The main solving big ridge regression problems

In [1]:
workspace()

In [45]:
using Plots

In [180]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [181]:
#Basic parameters
## Basic parameters
maxiter=10^8;
max_time = 10;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [182]:
dataset = "a9a"
prob = defineProb(dataset);

../data/a9a.jld
../data/a9a.jld


In [183]:
norm(prob.A*prob.xsol - prob.b)

4.459713089533306e-11

In [184]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [200]:
 s = options.sketchsize;
    rho = convert(Int64,floor(prob.n/s)); #hard coded density of rows
    ind = sample(1:prob.n,prob.n,replace=false); # shuffle all indices
    modns = mod(prob.n,s);
    divi = prob.n-modns;
    sold =s;
    if(modns!=0) 
        s= s+1;
    end
    indM = Array{Int64}(s,rho);# build a matrix indices with approx equally distributed number of indices over s rows
    indM[1:sold,:] =reshape(ind[1:divi], sold,rho);
    if(modns!=0)
        indM[s,1:modns] = ind[divi+1:end];
        indM[s,modns+1:end] =sample(1:prob.n, rho-modns,replace=false);
    end
    
    sigs = sample(1:2,prob.n,replace=true).*2.-3;

     SA = zeros(s,prob.n);
     Sb = zeros(s);

     for i =1:s
           SA[i,:] = sum(sigs[indM[i,:]].*prob.A[indM[i,:],:],1)/s;
           Sb[i] = sum(sigs[indM[i,:]].*prob.b[indM[i,:]])/s
     end
     SAS = zeros(s,s); 
      for i =1:s
          SAS[i,:] = sum(sigs[indM[i,:]]'.*SA[:,indM[i,:]],2)/s;
     end
     y = SAS\(SA*x-Sb);   # solving (S^TAS) y = (S^TAx-S^Tb)  
     for i =1:s #adding on S^T y
        x[indM[i,:]] = x[indM[i,:]]-sigs[indM[i,:]].*y[i];
     end

LoadError: DimensionMismatch("arrays could not be broadcast to a common size")

In [210]:
i =1;
sigs[indM[i,:]].*y[i]

24-element Array{Float64,1}:
 -0.31475
  0.31475
  0.31475
  0.31475
 -0.31475
  0.31475
 -0.31475
 -0.31475
  0.31475
  0.31475
 -0.31475
 -0.31475
  0.31475
  0.31475
 -0.31475
 -0.31475
 -0.31475
  0.31475
  0.31475
  0.31475
 -0.31475
  0.31475
 -0.31475
  0.31475

In [191]:
# Testing Rademarcher Sketch
method_name = "rademacher";# ROS
options.sketchsize =5;
options.maxiter=200;
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 4.90  |  2.66  | 0.0058 
 50  | 2.48  |  1.67  | 0.0114 
 75  | 1.48  |  1.05  | 0.0166 
100  | 1.11  |  0.70  | 0.0360 
125  | 0.88  |  0.52  | 0.0405 
150  | 0.78  |  0.38  | 0.0498 
175  | 0.71  |  0.41  | 0.0538 
200  | 0.66  |  0.44  | 0.0594 
  0.103245 seconds (44.16 k allocations: 36.432 MB, 4.41% gc time)


In [162]:
# Testing Rademarcher Sketch
method_name = "rademacher";# ROS
options.sketchsize =5;
options.maxiter=200;
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 4.09  |  3.44  | 0.0065 
 50  | 1.61  |  1.65  | 0.0181 
 75  | 0.96  |  1.02  | 0.0237 
100  | 0.65  |  0.62  | 0.0292 
125  | 0.47  |  0.52  | 0.0344 
150  | 0.36  |  0.35  | 0.0395 
175  | 0.30  |  0.68  | 0.0487 
200  | 0.26  |  0.27  | 0.0525 
  0.060893 seconds (43.81 k allocations: 36.396 MB, 15.88% gc time)


In [77]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =1; # Int(floor(log2(prob.n)));
options.maxiter=200;
options.skip_error_calculation =25;
@time  output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

flopsperiter: 247
Hadamard
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 26.22  |  15.76  | 0.2873 
 50  | 15.05  |  8.98  | 0.4753 
 75  | 12.14  |  7.24  | 0.6627 
100  | 9.28  |  5.99  | 0.8511 
125  | 6.00  |  4.39  | 1.0246 
150  | 5.57  |  4.23  | 1.2168 
175  | 5.12  |  3.55  | 1.4131 
200  | 4.56  |  3.41  | 1.6115 
  1.633492 seconds (15.71 M allocations: 1.651 GB, 23.07% gc time)


In [78]:
y = rand(prob.n) 
@time hada(y,1:prob.n);

  0.166638 seconds (116.06 k allocations: 6.144 MB)


In [62]:
# Testing countmin 
# options.max_time = 2; options.restol = 10.0^(-8); 
# options.sketchsize =5;
# options.skip_error_calculation =25;
# method_name = "countmin";# gradient descent
# output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

countmin
-------------------
It   | Error% | Residual |  Time   
-------------------


LoadError: UndefVarError: as not defined

In [71]:
# Testing CG 
options.max_time = 2; options.restol = 10.0^(-8); 
options.skip_error_calculation =25;
method_name = "CG";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 0.03  |  0.07  | 0.0467 
 50  | 0.00  |  0.06  | 0.0473 
 75  | 0.00  |  0.01  | 0.0479 


In [72]:
# Testing CD 
options.max_time = 10; options.restol = 10.0^(-8); options.maxiter=200;
method_name = "CD";# gradient descent
options.sketchsize = 5#floor((prob.n)^(1/2));
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 6.13  |  4.46  | 0.0451 
 50  | 3.73  |  3.35  | 0.0456 
 75  | 1.11  |  1.92  | 0.0461 
100  | 0.52  |  0.98  | 0.0466 
125  | 0.27  |  0.84  | 0.0471 
150  | 0.19  |  0.47  | 0.0476 
175  | 0.15  |  0.40  | 0.0481 
200  | 0.12  |  0.28  | 0.0485 


In [73]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [74]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

a9a


In [29]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];